In [14]:
import pandas as pd
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# Download required NLTK data
nltk.download('wordnet')
nltk.download('stopwords')

# Load dictionary and test data
dictionary_path = 'Dictionary 10.xlsx'
test_data_path = 'TEST DATA.csv'

# Read dictionary from excel file
dictionary_df = pd.read_excel(dictionary_path)

# Read test data from csv file
test_data_df = pd.read_csv(test_data_path)

# Initialize lemmatizer
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

# Preprocess text: lowercase, remove stopwords, and lemmatize
def preprocess(text):
    return ' '.join(
        lemmatizer.lemmatize(word)
        for word in text.lower().split()
        if word not in stop_words
    )

# Preprocess dictionary words
dictionary_df = dictionary_df.applymap(lambda x: preprocess(x) if isinstance(x, str) else x)

# Preprocess test data sentences
test_data_df['Text'] = test_data_df['Text'].astype(str).apply(preprocess)

# Aggregate text by Text-ID
aggregated_text_df = test_data_df.groupby('Text-ID')['Text'].apply(' '.join).reset_index()

# Initialize output dataframe with the necessary columns
output_df = aggregated_text_df[['Text-ID']].copy()

# Flatten the dictionary to create a unique vocabulary list
vocabulary = dictionary_df.values.flatten()
vocabulary = list(set([word for word in vocabulary if pd.notna(word)]))

# Initialize TfidfVectorizer with the unique vocabulary
vectorizer = TfidfVectorizer(vocabulary=vocabulary, lowercase=False)

# Fit and transform the aggregated text data
tfidf_matrix = vectorizer.fit_transform(aggregated_text_df['Text'])

# Convert the TF-IDF matrix to a DataFrame
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=vectorizer.get_feature_names_out())

# Merge the TF-IDF scores with the output DataFrame
output_df = pd.concat([output_df, tfidf_df], axis=1)

# Initialize dictionary count columns with zeros
behavior_columns = ["Security", "Conformity", "Tradition", "Benevolence", "Universalism",
                    "Self-Direction", "Stimulation", "Hedonism", "Achievement", "Power"]

# Initialize a DataFrame for counts
result_counts = output_df[['Text-ID']].copy()

# Function to count matches
def count_matches(sentence, words_list):
    sentence_words = set(sentence.lower().split())
    return sum(word in sentence_words for word in words_list)

# Update result_counts with counts based on aggregated text
for idx, row in aggregated_text_df.iterrows():
    sentence = row['Text']  # Aggregated sentence
    for column in behavior_columns:
        words_list = dictionary_df[column].dropna().str.lower().tolist()
        result_counts.at[idx, column] = count_matches(sentence, words_list)

# Binary DataFrame with same structure as result_counts
binary_counts = result_counts.copy()

# Initialize all count columns to 0 for binary
binary_counts[behavior_columns] = 0

# Iterate over each row to set the highest value to 1
for idx, row in result_counts.iterrows():
    highest_category = row[1:].idxmax()  # Find the column with the highest count
    binary_counts.at[idx, highest_category] = 1

# Save the result_counts (Word Counts) to Excel
output_path_counts = 'Word_Counts_t.xlsx'
result_counts.to_excel(output_path_counts, index=False, sheet_name='Counts')

# Save the binary_counts (Binary DataFrame) to Excel
output_path_binary = 'Word_Binary_t.xlsx'
binary_counts.to_excel(output_path_binary, index=False, sheet_name='Binary')

print("Files have been saved successfully.")


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\rites\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\rites\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
C:\Users\rites\AppData\Local\Temp\ipykernel_26124\3964184404.py:34: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  dictionary_df = dictionary_df.applymap(lambda x: preprocess(x) if isinstance(x, str) else x)


Files have been saved successfully.


In [19]:
import pandas as pd

# Load the provided Excel files
expert_binary_path = 'Expert_binary.xlsx'
word_binary_path = 'Word_Binary_t.xlsx'

expert_binary = pd.read_excel(expert_binary_path)
word_binary = pd.read_excel(word_binary_path)

# Merge the two dataframes on 'Text-ID'
merged_df = pd.merge(expert_binary, word_binary, on='Text-ID', suffixes=('_expert', '_word'))

# Create new columns
merged_df['Expert Category'] = merged_df.filter(regex='_expert$').idxmax(axis=1).str.replace('_expert', '')
merged_df['Word Category'] = merged_df.filter(regex='_word$').idxmax(axis=1).str.replace('_word', '')
merged_df['Match'] = merged_df['Expert Category'] == merged_df['Word Category']

# Adjust the calculation of match percentage to compare categories rather than words
def calculate_category_match_percentage(expert_category, word_category):
    return 100.0 if expert_category == word_category else 0.0

# Calculate Expert Match % and Word Match %
merged_df['Expert Match %'] = merged_df.apply(lambda row: calculate_category_match_percentage(row['Expert Category'], row['Word Category']), axis=1)
merged_df['Word Match %'] = merged_df.apply(lambda row: calculate_category_match_percentage(row['Word Category'], row['Expert Category']), axis=1)

# Select required columns with correct suffixes
expert_columns = [col for col in merged_df.columns if col.endswith('_expert')]
word_columns = [col for col in merged_df.columns if col.endswith('_word')]
category_columns = [col.replace('_expert', '') for col in expert_columns]

# Prepare the final dataframe
final_df = merged_df[['Text-ID'] + expert_columns + word_columns + ['Expert Category', 'Word Category', 'Match', 'Expert Match %', 'Word Match %']]

# Rename the columns for clarity
final_df.columns = ['Text-ID'] + [f'Expert_{col.replace("_expert", "")}' for col in expert_columns] + \
                   [f'Word_{col.replace("_word", "")}' for col in word_columns] + \
                   ['Expert Category', 'Word Category', 'Match', 'Expert Match %', 'Word Match %']

# Save the final dataframe to a new Excel file
output_path_final = 'Final_tfidf_Output.xlsx'
final_df.to_excel(output_path_final, index=False)

print(f"Final output saved to {output_path_final}")


Final output saved to Final_tfidf_Output.xlsx
